In [1]:
from watsonx import WatsonxAI

proxy = "proxy.us.ibm.com:8080"

wx = WatsonxAI()
wx.connect()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'iam.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [2]:
import io
import sys
import traceback
from IPython.core.interactiveshell import InteractiveShell
def call_tool(module_import, function_call, max_traceback_lines=5):
    # Create StringIO objects to capture standard output and error output
    output_stream = io.StringIO()
    error_stream = io.StringIO()

    # Get the current interactive shell
    shell = InteractiveShell.instance()

    # Redirect stdout and stderr
    old_stdout = sys.stdout
    old_stderr = sys.stderr
    sys.stdout = output_stream
    sys.stderr = error_stream

    output = None
    error = None

    # Using exec to import the module and evaluate the function call
    try:
        local_namespace = {}
        exec(module_import, local_namespace, local_namespace)  # Dynamically import the module
        print(f"|{function_call}|")
        exec(function_call, local_namespace, local_namespace)  # Evaluate the function call
    except ImportError as e:
        error = f"Import Error: {str(e)}"
    except Exception as e:
        # Capture the traceback for more detailed error information
        full_traceback = traceback.format_exc().strip()
        # Limit the traceback to the last 'max_traceback_lines' lines
        limited_traceback = "\n".join(full_traceback.splitlines()[-max_traceback_lines:])
        error = f"Unexpected Error: {str(e)}\n{limited_traceback}"
    finally:
        # Reset stdout to its original state
        sys.stdout = old_stdout
        sys.stderr = old_stderr

    # Get the output from the output_stream
    output = output_stream.getvalue().strip()
    error_output = error_stream.getvalue().strip()

    # Check the output or error
    if error:
        return f"Output: {output}", f"Error: {error}"
    elif error_output:
        return f"Output: {output}", f"Error Output: {error_output}"
    else:
        return f"Output: {output}", None

In [3]:
class Agent:
    name = "Agent"
    nextAgent = None
    debug = False
    model_id = ""
    system_prompt = ["You are a helpful agent."]
    functions = []
    tool_choice = None
    parallel_tool_calls = True
    input_names = []
    output_name = []
    prompt = ""

    def __init__(self,name,prompt,input_names, output_name, model_id=wx.GRANOTE_3_8B_INSTRUCT, debug=False):
        self.name = name
        self.prompt = prompt
        self.input_names = input_names
        self.output_name = output_name
        self.model_id = model_id
        self.debug = debug

    def run(self,context,history):
        print(f"{self.name} running")
        cur_prompt = self.prompt;
        for name in self.input_names:
            if name in context:
                cur_prompt = cur_prompt.replace(f"{{{name}}}",context[name])

        cur_prompt = cur_prompt.replace("{history}",str(history))
        if self.debug:
            print(cur_prompt)
        answer = wx.watsonx_gen(cur_prompt,wx.GRANOTE_3_8B_INSTRUCT).replace("```","")
        entry = f"""{{
                           agent:`{self.name}`,
                           input:`{context}`,
                           output:`{answer}`}}"""
        history.append(entry)
        if self.debug:
            print(entry)
        context[self.output_name] = answer
        return answer

planningAgent = Agent(
    name = "Planning Agent",
    prompt = """[INST]
please generate a concise and solid plan to satisfy the requirement provided.
-understand the requirement in detail.
-list out what knowlege you need to satisfy the requirement.
-think logically.
-think step by step.
-you trust the result of the tools more than your memory.
-you trust the knowledge provide more than your memory.
-leverage tool if possible.
-dont guess.
-please breakdown complex task to simple tasks.
-DONT generate result of the steps.
-**DONT** generate **Final Result:**
<<SYS>>
requirement: `{requirement}`
<</SYS>>
[/INST]plan:""",
    input_names = ["requirement"],
    output_name = "plan"
)

triageAgent = Agent(
    name = "Triage Agent",
    prompt = """[INST]
you are triager to determine who will be the next agent to handle the task.
please make your best judgement base on the history and agent available.
<<SYS>>
history: `{history}`
agents: {agents}
[/INST]agent:""",
    input_names = ["history","agents"],
    output_name = "next"
)

codingAgent = Agent(
    name = "Coding Agent",
    prompt = """[INST]
please generate python code that execute the plan to fulfill the requirement.
-ensure the python generated is executable.
-clean up if any unneccessary characters.
-review the problem in the code and the error if provided.
-prevent the error from the code if provided.
-you can use the tool provided directly without define it.
-understand the plan in detail. dont miss the steps.
-**DONT** use SaaS service which need API KEY.
-**DONT** catch error.
-raise error when hit problem.
-**DONT** throw SystemExit
-**DONT** generate main().
-**DONT** generate explanation.
-generate the code directly.
-generate the code only.
<<SYS>>
requirement: `{requirement}`
code with problem: `{code}`
plan: `{plan}`
error: `{error}`
tools: [
    get_hko_weather()->str,
    query_duckduckgo(query:str)->str,
    search_arxiv(query:str)->str,
    query_wikipedia(title:str)->str
]
<</SYS>>
[/INST]
```python
""",
    input_names = ["requirement","plan","error"],
    output_name = "code"
)

# reviseAgent = Agent(
#     name = "Revise Agent",
#     prompt = """[INST]
# revise the code provided.
# -review the problem in the code and the error.
# -prevent the error from the code.
# -you can use the tool provided directly without define it.
# -understand the plan in detail. dont miss the steps.
# -**DONT** use SaaS service which need API KEY.
# -**DONT** catch error.
# -raise error when hit problem. 
# -**DONT** throw SystemExit
# -**DONT** generate main().
# -**DONT** generate explanation.
# -generate the code directly.
# -generate the code only.
# <<SYS>>
# code: `{code}`
# plan: `{plan}`
# error: `{error}`
# tools: [
#     get_hko_weather()->str,
#     query_duckduckgo(query:str)->str,
#     search_arxiv(query:str)->str,
#     query_wikipedia(title:str)->str
# ]
# <</SYS>>
# [/INST]
# fixed code:```python""",
#     input_names = ["code","plan","error"],
#     output_name = "revised"
# )

executeAgent = Agent(
    name = "Execute Agent",
    prompt = "",
    input_names = [],
    output_name = ""
)

evalAgent = Agent(
    name = "Evaluate Agent",
        prompt = """[INST]classify if stdout include error.
-show sucess or fail only, but nothing else.
-show success if include no error. and end the generation.
-show fail if include error. and end the generation.
<<SYS>>
question: `{question}`
stdout: `{stdout}`
<</SYS>>
[/INST]success or fail:""",
    input_names = ["question","stdout"],
    output_name = "eval"
)

summaryAgent = Agent(
    name = "Summary Agent",
    prompt= """[INST]
please generate a complete sentence summary base on the answer provided only.
-dont use your common sense.
-only base on the answer provided.
-use simple english.
-dont guess.
<<SYS>>
requirement: `{requirement}`
answer: `{output}`
<</SYS>>
[/INST]answer summary:""",
    input_names = ["requirement","output"],
    output_name = "summary"
)
    
class AgenticPlaygroud:
    watsonxAI = None
    history = []
    context = {}
    agents = []
    activeAgent = None
    tools = []
    messages = []

    def __init__(self,watsonxAI):
        self.watsonxAI = watsonxAI

    def run(self,input):
        self.active_agent = Agent(self.model_id,requirement)
        while len(self.history) < self.maxTurns:
            requirement = ""
            output, vars, nextAgent = self.watsonxAI.watsonx_gen(input,self.history)
            self.history.append(input)
            self.history.append(output)
            self.context.update(vars)
            self.active_agent = nextAgent

    def agentic_prompt(self, requirement, max_turns=10,debug=False):
        self.context = {
            "requirement": requirement
        }
        self.history = []
        # Generate initial action and code
        planningAgent.run(self.context,self.history)
        
        # Loop until the review output is empty
        for turn in range(max_turns):
            print(f"turn #{turn+1} ---")
            # output = exec_with_output(code)
            codingAgent.run(self.context,self.history)
            self.context['output'], self.context['error'] = call_tool("",self.context['code'])
            if debug:
                print(f"#3: {self.context['output']}")
            
            if self.context['error'] is None:
                break
            
            if debug:
                print(f"error {self.context['error']}")
            # review = eval_output(requirement, output)
            # if debug:
            #     print(f"#4: {review}")
            
            # if "success" in review:  # Exit if review output is empty
            #     # print(f"#final result: {output}")
            #     break

        summaryAgent.run(self.context,self.history)
        return self.context['summary']

In [4]:
tools = """[
    get_hko_weather()->str,
    query_duckduckgo(query:str)->str,
    search_arxiv(query:str)->str,
    query_wikipedia(title:str)->str
]"""

In [ ]:
output = AgenticPlaygroud(wx).agentic_prompt("how is weather today in Hong Kong, base on HKO",debug=True)
print(output)

In [ ]:
output = AgenticPlaygroud(wx).agentic_prompt("what is the population in hong kong",debug=True)
print(output)

In [ ]:
output = AgenticPlaygroud(wx).agentic_prompt("what is stock price for IBM today")
print(output)

In [5]:
output = AgenticPlaygroud(wx).agentic_prompt("how many prime number with in 10000")
print(output)

Planning Agent running
turn #1 ---
Coding Agent running
Summary Agent running
 There are 1229 prime numbers within 10000.


In [6]:
output = AgenticPlaygroud(wx).agentic_prompt("how many R in straberry, not regarding to upper lower case")
print(output)

Planning Agent running
turn #1 ---
Coding Agent running
Summary Agent running
 There are 3 R's in the word "strawberry".


In [7]:
output = AgenticPlaygroud(wx).agentic_prompt("in math, compare 9.11 with 9.8")
print(output)

Planning Agent running
turn #1 ---
Coding Agent running
Summary Agent running
 9.11 is not greater than 9.8.


In [8]:
output = AgenticPlaygroud(wx).agentic_prompt("tell me Hong Kong time")
print(output)

Planning Agent running
turn #1 ---
Coding Agent running
turn #2 ---
Coding Agent running
Summary Agent running
 The current time in Hong Kong is 2024-10-21 22:11:54 HKT+0800.


In [9]:
output = AgenticPlaygroud(wx).agentic_prompt("tell me about Paris")
print(output)

Planning Agent running
turn #1 ---
Coding Agent running
Summary Agent running
 The function tells about Paris by querying Wikipedia for geographical,, historical, cultural, and tourist information.


In [10]:
output = AgenticPlaygroud(wx).agentic_prompt("give me answer of 320*232 / 2322, keep 2 decimal point only.")
print(output)

Planning Agent running
turn #1 ---
Coding Agent running
Summary Agent running
 The result of 320*232 divided by 2322 is 31.97.


In [11]:
output = AgenticPlaygroud(wx).agentic_prompt("tell me the biggest planet in solar system")
print(output)

Planning Agent running
turn #1 ---
Coding Agent running
Summary Agent running
 The biggest planet in the solar system is Jupiter.


In [ ]:
def clock():
    print("12pm")
    return "12pm"

def calculate(expression):
    return eval(expression)

prompt = f"""[INST]
tell me 3434 * 21 / 32
-for any math question, dont base on memory. ride on tool.
-tell what you confidence to answer, dont guess.
-generate the python code to use tool in need.
-only use the tool provided.
-dont repeat.
-generate the answer or code only.
<<SYS>>
tools: [
    clock()->str #get time,
    abc(code:str)->str #get stock price, 
    calculator(expression:str)->str,
    browser(url:str)->str
    ]
<</SYS>>
[/INST]answer:
"""

answer = wx.watsonx_gen(prompt,wx.LLAMA_3_70B_INSTRUCT)
print(answer)
out = call_tool("",answer.strip())
print(out)

In [ ]:
print(320*232 / 2322)

In [ ]:
def clock():
    print("12pm")
    return "12pm"

def ask_granite(question):
    prompt = f"""<|system|>
-tell what you confidence to answer only, dont guess.
-select the right tool.
-dont base on memory. call the right tool in contenxt.
[TOOLS]
tools: [
    clock()->str #get time,
    abc(code:str)->str #get stock price, 
    calculator(expression:str)->str # do math,
    browser(url:str)->str #find informatino
    ]
[/TOOLS]
-generate the python code to use tool in need.
-only use the tool provided.
-dont repeat.
-generate the answer or code only.
-dont generate result.
-dont generate Calculation steps.
-generate the code only.
<|user|>
{question}
<|assistant|>
```python"""

    answer = wx.watsonx_gen(prompt,wx.GRANITE_13B_CHAT_V2)
    return answer

answer = ask_granite("tell me the content of cnn.com today")
print(answer)

answer = ask_granite("tell me 320*232 / 2322")
print(answer)

answer = ask_granite("tell me about paris")
print(answer)